In [14]:
def read_dimacs_file(filename):
    clauses = []
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('c') or line.startswith('p'):
                continue
            clause = list(map(int, line.split()))
            if not clause:
                continue
            if clause[-1] == 0:
                clause = clause[:-1]
            clauses.append(clause)
    return clauses

In [1]:
class Graph:
    def __init__(self):
        self.graph = {}
    
    def add_node(self, variable, value):
        self.graph[variable] = {"value": value, "source": []}
    
    def add_source_edge(self, variable, source_literal):
        if variable in self.graph:
            self.graph[variable]["source"].append(source_literal)
    
    def __getitem__(self, key):
        return self.graph.get(key, {})

In [2]:
def check_unit_clause(clauses, assignment):
    """Checks for unit clauses and updates the assignment vector."""
    for clause in clauses:
        if len(clause) == 1:
            variable = abs(clause[0])
            if clause[0] > 0:
                assignment[variable] = True
                clauses.remove(clause)
            else:
                assignment[variable] = False
                clauses.remove(clause)
    return clauses, assignment

In [3]:
def simplify(clauses, assignment):
    new_clauses = []
    for clause in clauses:
        new_clause = []
        satisfied = False
        for literal in clause:
            variable = abs(literal)
            if variable in assignment:
                if (literal > 0 and assignment[variable]) or \
                   (literal < 0 and not assignment[variable]):
                    satisfied = True
                    break
                continue
            else:
                new_clause.append(literal)
        if not satisfied:
            new_clauses.append(new_clause)
    return new_clauses

In [4]:
def find_pure_literal(clauses, assignment):
    literals = {}
    mono_literals = {}
    for clause in clauses:
        for literal in clause:
            variable = abs(literal)
            if variable not in assignment:
                literals[literal] = literals.get(literal, 0) + 1
    for literal in literals:
        if -literal not in literals:
            mono_literals[literal] = literals[literal]
    return mono_literals

In [5]:
def unassigned_literal(clauses):
    literals = {}
    for clause in clauses:
        for literal in clause:
            variable = abs(literal)
            if variable not in literals:
                literals[variable] = [0, 0]
            if literal > 0:
                literals[variable][0] += 1
            else:
                literals[variable][1] += 1
    return dict(sorted(literals.items(), key=lambda item: sum(item[1]), reverse=True))

In [28]:
def dpll_cdcl(clauses, assignment={}, iterations=0, decision_level=0, implication_graph=None):
    if implication_graph is None:
        implication_graph = Graph()

    pure_literals = find_pure_literal(clauses, assignment)
    for literal in pure_literals:
        variable = abs(literal)
        value = literal > 0
        if variable not in assignment:
            assignment[variable] = value
            implication_graph.add_node(variable, value)

    new_clauses = simplify(clauses, assignment)

    if not new_clauses:
        return assignment, iterations

    if any(not clause for clause in new_clauses):
        # Simulated clause learning: add a new learned clause
        learned_clause = [-lit for lit in assignment if assignment[lit]]
        if learned_clause:
            clauses.append(learned_clause)
        return None, iterations

    stats = unassigned_literal(new_clauses)
    if not stats:
        return assignment, iterations

    priority_var = next(iter(stats))
    new_assignment = assignment.copy()
    value = stats[priority_var][0] >= stats[priority_var][1]
    new_assignment[priority_var] = value
    implication_graph.add_node(priority_var, value)

    new_clauses_true = simplify(new_clauses, new_assignment)
    updated_clauses, unit_assignment = check_unit_clause(new_clauses_true, new_assignment)
    result_true = dpll_cdcl(updated_clauses, unit_assignment, iterations + 1, decision_level + 1, implication_graph)
    if result_true:
        return result_true

    new_assignment[priority_var] = not value
    implication_graph.add_node(priority_var, not value)

    new_clauses_false = simplify(new_clauses, new_assignment)
    updated_clauses, unit_assignment = check_unit_clause(new_clauses_false, new_assignment)
    result_false = dpll_cdcl(updated_clauses, unit_assignment, iterations + 1, decision_level + 1, implication_graph)
    if result_false:
        return result_false

    return None, iterations

# Load clauses from DIMACS file
filename = 'quinn.cnf'  # Replace with actual path
test_clauses = read_dimacs_file(filename)
test_assignment = {}
result, iterations = dpll_cdcl(test_clauses, test_assignment)
print("Resulting Assignment:", result)
print("Iterations:", iterations)

Resulting Assignment: None
Iterations: 3
